In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pathsDict = {'Kroeber':'Kroeber texts.xlsx',
 'Reichard' : 'Reichard texts.xlsx',
 'Teeter':'Teeter texts.xlsx'}

In [3]:
wiyotData = dict()
for k,v in pathsDict.items():
    print(v)
    wiyotData[k] = pd.read_excel(f'../data/{v}',None)

Kroeber texts.xlsx
Reichard texts.xlsx


/opt/miniconda3/envs/S3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Teeter texts.xlsx


In [4]:
def processLine(x):
    try:
        x = '\t'.join(x.dropna()).split('\t')
        return x
    except:
        return None
    
getlowercase = lambda x: x if x.isupper() else x.lower()    

In [5]:
def processWiyotFormat(author,source):
    data = wiyotData[author][source]
    alldictionaries = list()
    for i in range(0,len(data),3):
        wiyotPhrase = data.iloc[i]
        englishPhrase = data.iloc[i+1]
        ####
        wiyotPhrase = processLine(wiyotPhrase)
        ### Do some extra processing to the english phrase
        englishPhrase = processLine(englishPhrase)
        englishPhrase = [getlowercase(x) for x in englishPhrase]
        eng2wiyotDF = pd.DataFrame([englishPhrase,wiyotPhrase]).T    
        eng2wiyotDF.columns = ['english','wiyot']
        eng2wiyotDF['author'] = author
        eng2wiyotDF['source'] = source
        alldictionaries.append(eng2wiyotDF)
    return alldictionaries

In [20]:
wiyotData[author][story]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,Kwhali,yo,to,lól,it.,NaN,NaN
1,NaN,Here,INT,PROG,go,2SBJ.,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Yíl,"woł,",kwhali,to,lól.,NaN,NaN
4,NaN,1SG,also,here,PROG,go[1SBJ].,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Čawo,to,tu,dakw,ih,khu,kúhwik.
7,NaN,DEM,DEF,PL,be.so,NMLZ.P,2POSS,family.
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,To,tu,wénutw,ił.,NaN,NaN,NaN


In [6]:
authors = list(wiyotData.keys())
sourcesByAuthor = dict()
for author in authors:
    print(author)
    sourcesByAuthor[author] = list(wiyotData[author].keys())

Kroeber
Reichard
Teeter


In [16]:
processedtext = []
for author in authors:
    for story in sourcesByAuthor[author]:
        try:
            tmp = processWiyotFormat(author,story)
            processedtext.append(tmp[0])
        except:
            print(f'skipping: {author}:,{story}')

skipping: Reichard:,2. Wolf's Home


In [12]:
import pickle as pkl

In [18]:
pkl.dump(processedtext,open('../results/individual_english2wiyottables.pkl','wb'))

In [10]:
len(processedtext)

135

In [ ]:
eng2wiyotDFAll = pd.concat(alldictionaries)

In [ ]:
uniqueEnglishWords = list(set(eng2wiyotDFAll.english))

In [ ]:
eng2wiyot = dict()
for engWord in uniqueEnglishWords:
    """
    For each english word, get all the corresponding wiyot words and create a dictionary.
    This takes care of differences in spelling
    """
    wiyotMatch = eng2wiyotDFAll[eng2wiyotDFAll.english==engWord]['wiyot']
    wiyotMatch = list(wiyotMatch)
    wiyotMatch = list(set([_.lower() for _ in wiyotMatch]))
    eng2wiyot[engWord] = wiyotMatch

In [ ]:
eng2wiyot['put']

In [ ]:
englishWordFreq = eng2wiyotDFAll['english'].value_counts().reset_index()
englishWordFreq.columns = ['word','frequency']

In [ ]:
dat = englishWordFreq[englishWordFreq['frequency'] > 1]

In [ ]:
dat = englishWordFreq[englishWordFreq['frequency'] > 1]
plt.figure(figsize=(3,7))
plt.title('Frequency of words')
sns.barplot(y='word',x='frequency',data=dat,color='skyblue')
plt.show()

In [22]:
data = wiyotData[author][story]

In [25]:
alldictionaries = list()
for i in range(0,len(data),3):
    wiyotPhrase = data.iloc[i]
    englishPhrase = data.iloc[i+1]
    ####
    wiyotPhrase = processLine(wiyotPhrase)
    ### Do some extra processing to the english phrase
    englishPhrase = processLine(englishPhrase)
    englishPhrase = [getlowercase(x) for x in englishPhrase]
    eng2wiyotDF = pd.DataFrame([englishPhrase,wiyotPhrase]).T    
    eng2wiyotDF.columns = ['english','wiyot']
    eng2wiyotDF['author'] = author
    eng2wiyotDF['source'] = story

In [29]:
wiyotPhrase

['Čawókš,', 'yil', 'ał.']

In [28]:
englishPhrase

['good.bye', '1SG', 'ADVS.']